# Water Quality Parameters - Statistics

In [1]:
# Styling notebook

# System
import os
import sys

# Import scripts libraries for the project
sys.path.append('./src/python')

# Import the function to update the notebook style
from nbConfig import (css_styling)

css_styling()

Import the libraries needed for the analysis

In [2]:
# Data
import math
import numpy as np
import pandas as np

# Spatial Data
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats, point_query

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns